# 检测假新闻

将借助Sklearn在数据集上建立一个TfIdfVectorizer, 然后初始化一个PassiveAggressive
CLassifier并将拟合. 在最后, 准确度分数和相关性矩阵将表明模型的效果.

# TfIdfVectorizer
**TF(Term Frequency)**: 是一个单词在文章中出现的次数. 越高意味着一个单词相比其他
单词出现的次数更多, 因此当这个单词是搜索词的一部分的时候, 这篇文章将是良好的搜索
结果;
**IDF(Inverse Document Freeuency)**: 在一篇文章中出现了很多次的单词也在其他的中
出现了很多次, 可能是不相关的. IDF就是一种判断一个单词在整个群中的重要性的方法;

TfIdfVectorizer可以将一系列的原文档转为TF-IDF特征矩阵.

# PassiveAggressiveClassifer
Passive Aggressive算法(被动攻击性算法)是一组在线学习算法. 这样的算法对于一个正确
的分类输出保留保持, 对于错误的分类变得激进, 进行更新和调整. 不像大多数其他的算
法, 该算法不收敛. 他的目的是造成更新来修复损失, 在权重向量中引起非常小的改变.

# 假新闻数据集介绍
本次的数据集为`news.csv`, 其形状为$7796\times 4$. 第一列标识新闻, 第二盒第三列为
标题和内容, 第四列是标识新闻是`REAL`还是`FAKE`的标签. 这个数据集大小为**29.2MB**.

In [1]:
# import libraries
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

## Read the data into a DataFrame

In [4]:
# read the data
df = pd.read_csv("../dataset/news.csv")

# get shape and head
print(df.shape)
df.head()

(6335, 4)


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
# get labels from the DataFrame
labels=df.label
labels.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [5]:
# split the dataset into training and testing sets
X_train,X_test,y_train,y_test=train_test_split(df['text'],labels,test_size=0.2,random_state=7)

## 初始化一个 TfidfVectorizer

使用英语中的停止单词和最大的文章频率为 0.7(有更高的文章频率的单词会被丢弃). 停止
词是在语言中在处理自然语言数据之前需要被拆选出去的最频繁的单词. 一个TfidfVectorizer可以把一组文章转变成TF-IDF特征矩阵.


In [6]:
# Initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english',max_df=0.7)

# Fit and transform train set, transform test set
tfidf_train=tfidf_vectorizer.fit_transform(X_train)
tfidf_test=tfidf_vectorizer.transform(X_test)

# 初始化一个PassiveAggressiveClassifier
将其在tfidf_train和y_train上拟合

In [8]:
# initialize a Passive Aggressive Classifier
pac = PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train, y_train)

# predict on the test set and calculate accuracy
y_pred = pac.predict(tfidf_test)
score = accuracy_score(y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(round(score * 100, 2)))

Accuracy: 92.74%


最后, 打印一个混淆矩阵来查看真假阳性和阴性的数量

In [9]:
# build confusion matrix
confusion_matrix(y_test,y_pred,labels=['FAKE','REAL'])

array([[588,  50],
       [ 42, 587]])

在这个模型中, 有588个真阳性, 587个真阴性, 42个假阴性, 50个假阳性.

# 总结
借助TfidfVectorizer, PassiveAggressiveClassifier来在新闻数据集上拟合模型, 获取了92.74%的准确度.